In [17]:
%matplotlib inline

from pathlib import Path
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import savReaderWriter as srw

DATA_EXT = (Path(os.getcwd()) / os.pardir / 'data' / 'external').resolve()

In [2]:
import sys
sys.path.append(os.path.join(os.getcwd(), os.pardir, 'src'))

%load_ext autoreload

%autoreload 2
from visualization.visualize import *

In [4]:
who_df = pd.read_csv(Path("../data/interim/calc_cols_added.csv"), index_col=0)
who_df.head(2)

,Iso Code,Country Name,WHO Region,Year,Vaccine Type,Admin1,Admin2,DenomType,Denominator,Numerator,Coverage,recalc_numerator,available_admin,normalized_country,indicator,group,vaccine,timing
0,AFG,Afghanistan,EMRO,2016,BCG,NaN,Aab Band,1.0,1266.0,NaN,51.801245,655.803767,Aab Band,afghanistan,1,BCG,BCG,1st_birth
1,AFG,Afghanistan,EMRO,2016,BCG,NaN,Aab Kamari,1.0,4599.0,NaN,94.674670,4354.088093,Aab Kamari,afghanistan,1,BCG,BCG,1st_birth


In [14]:
# These countries appear in both the MICS and the WHO datasets
overlaps = [
    'Benin',
    'Cameroon',
    'Cuba',
    'Dominican Republic',
    'El Salvador',
    'Guyana',
    'Kazakhstan',
    'Kyrgyzstan',
    'Malawi',
    'Mali',
    'Mexico',
    'Nepal',
    'Paraguay',
    'Sudan',
    'Swaziland',
    'Turkmenistan',
    'Viet Nam',
    'Zimbabwe',
]

# some MICS regions are just subsets of a country, so we
# can only analyze the ones that appear in both datasets
partial_matches = {
    'Kenya (Bungoma County)': 'Kenya',
    'Kenya (Kakamega County)': 'Kenya',
    'Kenya (Turkana County)': 'Kenya',
    'Pakistan (Punjab)': 'Pakistan',
    'Pakistan (Sindh)': 'Pakistan',
}

# Some MICS countries have their names written differently
# Thus maps the MICS version as the key to the WHO version
# as the value
overlap_map = {
    'Guinea Bissau': 'Guinea-Bissau',
    'Sao Tome and Principe': 'Sao Tome & Principe',
}

# Load MICS Survey data

In [15]:
mics_countries = []
for f in (DATA_EXT / 'MICS').iterdir():
    if f.name.startswith('.'):
        continue
        
    country = f.parts[-1].split("_")[0]
    mics_countries.append(country)
        
    for spss_file in f.glob('*SPSS Datasets/*.sav'):
        pass

In [18]:
# load with sav reader/writier
def spss_to_csv(input_filepath):
    """ Runs data processing scripts to turn raw data from (../raw) into
        cleaned data ready to be analyzed (saved in ../processed).
    """
    # load metadata
    with srw.SavHeaderReader(input_filepath, ioUtf8=True) as header:
        metadata = header.all()
        value_replacements = metadata.valueLabels

        nan_replacements = dict()
        for k, v in metadata.missingValues.items():
            if v and 'values' in v:
                nan_values = dict()
                if isinstance(v['values'], list):
                    for nan_val in v['values']:
                        nan_values[nan_val] = np.nan
                else:
                    nan_values[v['values']] = np.nan

                nan_replacements[k] = nan_values

        questions = metadata.varLabels

    with srw.SavReader(input_filepath, ioUtf8=True) as reader:
        header = reader.header
        records = reader.all()

    df = pd.DataFrame(records, columns=header)
    df.replace(value_replacements, inplace=True)
    df.replace(nan_replacements, inplace=True)

    return df, questions
    
    
#     df.to_csv(output_filepath)

#     questions_file = Path(output_filepath).with_suffix('.json')
#     with open(questions_file, 'w') as qf:
#         json.dump(questions, qf)

        
#     # upsample the survey for a "representative" sample for analysis
#     rng = np.random.RandomState(12345)

#     w_col = 'Weight' if 'Weight' in df.columns else 'weight'

#     smpl = rng.choice(df.index, 240000, p=df[w_col] / df[w_col].sum())
#     df_resampled = df.loc[smpl, :]

#     output_path = Path(output_filepath)
#     df_resampled.to_csv(output_path.with_name(output_path.stem + '_upsampled.csv'))



In [19]:
spss_to_csv(Path("../data/external/MICS/Benin_MICS5_Datasets/Benin MICS 2014 SPSS Datasets/hh.sav"))

OSError: dlopen(/Users/bull/anaconda3/envs/subnational-vaccines/lib/python3.6/site-packages/savReaderWriter/spssio/macos/libicuuc48.1.dylib, 6): Library not loaded: @executable_path/../lib/libicudata48.1.dylib
  Referenced from: /Users/bull/anaconda3/envs/subnational-vaccines/lib/python3.6/site-packages/savReaderWriter/spssio/macos/libicuuc48.1.dylib
  Reason: image not found

In [ ]:
# subset WHO to just the years / countries that are relevant and match the WHO data

In [ ]:
# apply country birth rate to denominator for the difference between MICS survey year
# and the WHO data year (which is 2016)

In [15]:
!tree {(DATA_EXT / 'MICS')}

/Users/bull/subnational-vaccines/data/external/MICS
├── Benin_MICS5_Datasets
│   ├── Benin\ MICS\ 2014\ SPSS\ Datasets
│   │   ├── bh.sav
│   │   ├── ch.sav
│   │   ├── fg.sav
│   │   ├── hh.sav
│   │   ├── hl.sav
│   │   ├── mm.sav
│   │   ├── mn.sav
│   │   ├── tn.sav
│   │   └── wm.sav
│   └── Read\ me_Benin.txt
├── Cameroon_MICS5_Datasets
│   ├── Cameroon\ MICS\ 2014\ SPSS\ Datasets
│   │   ├── bh.sav
│   │   ├── ch.sav
│   │   ├── gats.sav
│   │   ├── hh.sav
│   │   ├── hl.sav
│   │   ├── mn.sav
│   │   ├── tn.sav
│   │   └── wm.sav
│   └── Read\ me_Cameroon.txt
├── Cuba_MICS5_Datasets
│   ├── Cuba\ MICS\ 2014\ SPSS\ Datasets
│   │   ├── ch.sav
│   │   ├── hh.sav
│   │   ├── hl.sav
│   │   ├── mn.sav
│   │   └── wm.sav
│   └── Read\ me_Cuba.txt
├── Dominican\ Republic_MICS5_Datasets
│   ├── Dominican\ Republic\ MICS\ 2014\ SPSS\ Datasets
│   │   ├── bh.sav
│   │   ├── ch.sav
│   │   ├── hh.sav
│   │   ├── hl.sav
│   │   └── wm.sav
│   └── Read\ me_Dominican\ Republic.txt
├── Egypt